Implenntation of Transformer Arch 

<img src="image.png" width="500" height="600">



In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# from utils import utils

In [2]:
def initialize_weight(x):
    nn.init.xavier_uniform_(x.weight)
    if x.bias is not None:
        nn.init.constant_(x.bias, 0)
        
        
class FeedForwardNetwork(nn.Module):
     def __init__(self, hidden_size, filter_size, dropout_rate):
        super(FeedForwardNetwork, self).__init__()

        self.layer1 = nn.Linear(hidden_size, filter_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.layer2 = nn.Linear(filter_size, hidden_size)

        initialize_weight(self.layer1)
        initialize_weight(self.layer2)
        
     def forward(self,x):
        x= self.layer1(x)
        x= self.relu(x)
        x= self.dropout(x)
        x= self.layer(x)
        
        return x

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_size, dropout_rate, head_size=8):
        super(MultiHeadAttention,self).__init__()
        
        self.head_size = head_size
        self.att_size = att_size = hidden_size // head_size
        self.scale = att_size ** -0.5
        
        self.linear_q = nn.Linear(hidden_size,head_size*att_size, bias = False)
        self.linear_k = nn.Linear(hidden_size,head_size*att_size, bias = False)
        self.linear_v = nn.Linear(hidden_size,head_size*att_size, bias = False)
        
        initialize_weight(self.linear_q)
        initialize_weight(self.linear_k)
        initialize_weight(self.linear_v)
        
        self.att_dropout = nn.Dropout(dropout_rate)
        self.output_layer = nn.Linear(head_size*att_size,hidden_size,bias = False)
        
        initialize_weight(self.output_layer)
    
    def forward(self,q,k,v,mask, cache = None):
        return x 
        
    
        
        

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self,hidden_size,filter_size,dropout_rate):
        super(EncoderLayer,self).__init__()
        
        